In [262]:
import geopandas as gpd
import pandas as pd
import json
import math
import statistics

Params

In [263]:
STATE_NAME = "arizona"

if(STATE_NAME == "arizona"):
    SEATS = 9
    STATE_NAME_UPPER = "Arizona"
    MMD_PATTERN = [[3, 3, 3], [4, 5]]
    average_mmd_id = 4895
elif(STATE_NAME == "texas"):
    SEATS = 38
    STATE_NAME_UPPER = "Texas"
    MMD_PATTERN = [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4], [3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5], [3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5], [3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3, 5, 5, 5, 5], [3, 3, 3, 3, 3, 4, 4, 5, 5, 5], [3, 3, 3, 3, 4, 4, 4, 4, 5, 5], [3, 3, 3, 4, 4, 4, 4, 4, 4, 5], [3, 3, 3, 4, 5, 5, 5, 5, 5], [3, 3, 4, 4, 4, 4, 4, 4, 4, 4], [3, 3, 4, 4, 4, 5, 5, 5, 5], [3, 4, 4, 4, 4, 4, 5, 5, 5], [3, 5, 5, 5, 5, 5, 5, 5], [4, 4, 4, 4, 4, 4, 4, 5, 5], [4, 4, 5, 5, 5, 5, 5, 5]]
    average_mmd_id = 4895
elif(STATE_NAME == "utah"):
    SEATS = 4
    STATE_NAME_UPPER = "Utah"
    MMD_PATTERN = [[4]]
    average_mmd_id = 4895
else:
    raise Exception("Invalid State")

Importing Data

In [264]:
with open('data/'+STATE_NAME_UPPER+'/ensamble/smd_ensemble_'+STATE_NAME_UPPER+'.json') as json_file:
    ensamble_ids = json.load(json_file)
with open('data/'+STATE_NAME_UPPER+'/ensamble/mmd_ensemble_'+STATE_NAME_UPPER+'.json') as json_file:
    mmd_ensamble_ids = json.load(json_file)
with open('data/'+STATE_NAME_UPPER+'/ensamble/mmd_elections_'+STATE_NAME_UPPER+'.json') as json_file:
    mmd_election = json.load(json_file)
with open('data/'+STATE_NAME_UPPER+'/ensamble/mmd_safe_dists_'+STATE_NAME_UPPER+'.json') as json_file:
    mmd_safe_dists = json.load(json_file)
precincts = gpd.read_file('data/'+STATE_NAME_UPPER+'/'+STATE_NAME+'Geo.geojson')
precincts['asianPop'] = precincts['asainPop']
precincts = precincts.drop(['asainPop'], axis = 1)
display(type(ensamble_ids))
display(len(ensamble_ids))
display(precincts.head())

dict

10000

,index,geoid,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,precinct,demVotes,repVotes,totalVotes,district,geometry,asianPop
0,0,04015000209,1857,16,1329,372,14,dolan springs,0,0,0,9,"POLYGON ((-12767174.951 4252756.034, -12767132...",15
1,1,04015000211,41878,553,28420,9960,346,bullhead city,0,0,0,9,"POLYGON ((-12764337.461 4225677.685, -12764331...",585
2,2,04015000221,3175,18,2228,799,45,littlefield,0,0,0,9,"POLYGON ((-12696056.514 4439130.190, -12695993...",13
3,3,04015000223,276,6,66,16,178,moccasin,0,0,0,2,"POLYGON ((-12560122.887 4439178.791, -12559487...",1
4,4,04015000216,12214,95,9647,1565,157,kingman north,0,0,0,9,"POLYGON ((-12702961.835 4201596.146, -12702947...",87


Constants and Important Variables

In [265]:
ensamble = dict()
mmd_ensamble = dict()
DEMOCRAT_VOTES = precincts['demVotes'].sum()
REPUBLICAN_VOTES = precincts['repVotes'].sum()
WHITE_POPULATION = precincts['caucasianPop'].sum()
BLACK_POPULATION = precincts['africanAmericanPop'].sum()
LATINO_POPULATION = precincts['latinoPop'].sum()
ASIAN_POPULATION = precincts['asianPop'].sum()
NATIVE_AMERICAN_POPULATION = precincts['nativeAmericanPop'].sum()
TOTAL_POPULATION = precincts['totalPop'].sum()

display(type(ensamble_ids['0']))
display(ensamble_ids['0'])

dict

{'9': [0,
  1,
  2,
  1536,
  4,
  5,
  6,
  7,
  1537,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  1296,
  1299,
  1306,
  1314,
  1315,
  1316,
  1317,
  1062,
  1318,
  1319,
  1320,
  1321,
  1534,
  1072,
  1336,
  1341,
  1093,
  1094,
  1095,
  1286,
  1097,
  840,
  844,
  77,
  78,
  79,
  80,
  1101,
  1107,
  1109,
  1112,
  1113,
  1118,
  1377,
  1380,
  1292,
  1127,
  1293,
  1386,
  876,
  877,
  1403,
  432,
  434,
  435,
  680,
  681,
  682,
  1195,
  684,
  685,
  1199,
  688,
  689,
  690,
  691,
  692,
  693,
  694,
  695,
  696,
  1202,
  1203,
  946,
  947,
  437,
  438,
  439,
  441,
  442,
  443,
  444,
  445,
  1221,
  446,
  955,
  450,
  451,
  454,
  455,
  459,
  1232,
  466,
  467,
  985,
  1245,
  1246,
  989,
  1000,
  1266,
  1270,
  1272,
  1530,
  1531,
  1532,
  1533,
  1278,
  1535],
 '2': [3,
  8,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  4

Current District Plan Generation

In [266]:
def map_to_district(precinct_id, plan_id):
    plan = ensamble_ids[plan_id]
    for district_id in plan:
        if(precinct_id in plan[district_id]):
            return int(district_id)
    return -1

In [267]:
def is_opportunity(total_pop ,african_american_pop, latino_pop, asain_pop, native_american_pop):
    if(african_american_pop / total_pop >= .5):
        return True
    if(latino_pop / total_pop >= .5):
        return True
    if(asain_pop / total_pop >= .5):
        return True
    if(native_american_pop / total_pop >= .5):
        return True
    return False

SMD Ensamble Dataframe Generation

In [268]:
for plan_id in ensamble_ids:
    district_plan = precincts.copy(deep = True)
    district_plan['district'] = district_plan['index'].apply(lambda id: map_to_district(id, plan_id))
    district_plan.drop(["index", "precinct", "geometry"], axis = 1, inplace=True)
    district_plan = district_plan.groupby('district').aggregate(sum)
    ensamble[int(plan_id)] = district_plan

In [269]:
display(len(ensamble))
display(ensamble[0])
display(ensamble[5])
display(ensamble[10])
display(ensamble[100])
display(ensamble[1000])
display(ensamble[5000])
display(ensamble[9999])

10000

,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,784753,24880,546099,130424,12745,221755,219701,441456,35540
2,814640,17073,451607,139429,162861,88473,85697,174170,9746
3,817560,82899,161739,509125,16141,176335,51612,227947,21379
4,783490,41115,431105,210770,17475,190649,158102,348751,43009
5,795784,27945,534214,139641,6176,163805,242745,406550,47325
6,778797,26006,490754,193530,6937,201210,181421,382631,23152
7,793281,29736,233495,467261,23700,165193,104822,270015,15315
8,789363,34037,512649,163596,8086,169922,236598,406520,34785
9,793834,33470,454885,238477,9809,93476,121885,215361,18586


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,784753,24880,546099,130424,12745,221755,219701,441456,35540
2,814640,17073,451607,139429,162861,88473,85697,174170,9746
3,784560,36264,493712,174238,8142,162906,228742,391648,34671
4,792330,43163,424347,215195,17732,198015,157352,355367,50363
5,786944,25897,540972,135216,5919,156439,243495,399934,39971
6,798925,39472,446241,244527,10153,118281,136265,254546,20797
7,801044,27853,470901,230698,9071,214569,184200,398769,25348
8,776468,21558,281036,416641,21300,133775,95244,229019,11095
9,811838,81001,161632,505885,16007,176605,51887,228492,21306


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,782923,39463,457804,210840,9137,154947,210833,365780,30797
2,814640,17073,451607,139429,162861,88473,85697,174170,9746
3,803643,23859,564278,124485,12160,222028,231565,453593,41390
4,791572,44046,414335,223116,19714,196394,152455,348849,49360
5,786944,25897,540972,135216,5919,156439,243495,399934,39971
6,798925,39472,446241,244527,10153,118281,136265,254546,20797
7,801044,27853,470901,230698,9071,214569,184200,398769,25348
8,776468,21558,281036,416641,21300,133775,95244,229019,11095
9,795343,77940,189373,467301,13615,185912,62829,248741,20333


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,806953,54522,383686,256505,18157,207159,157816,364975,54709
2,786756,31905,385380,298554,12355,213438,148941,362379,24165
3,790676,31815,494377,198201,11541,120750,172213,292963,17408
4,783187,24814,539331,144604,8915,149282,239418,388700,26813
5,787496,26728,541801,133151,12649,203830,224172,428002,37902
6,798839,42608,415520,261282,8810,164820,189296,354116,35222
7,810949,30450,343974,365812,27398,136572,127664,264236,12381
8,797912,66682,257999,394790,18105,194235,72073,266308,29780
9,788734,7637,454479,139354,146000,80732,70990,151722,10457


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,774727,53508,350303,291604,13107,166712,139296,306008,31642
2,789849,29597,369528,323960,15152,200853,154529,355382,18884
3,798719,34598,493807,171664,21532,155492,220223,375715,38849
4,777267,37147,438540,223051,14329,209922,163870,373792,31340
5,784591,16644,433344,213800,69930,172603,140746,313349,17681
6,809673,32151,545489,156019,6699,173910,257302,431212,30867
7,812340,7719,458863,210527,95440,28626,30862,59488,8850
8,817624,37979,467546,212111,14740,181748,181183,362931,43411
9,786712,67818,259127,389517,13001,180952,114572,295524,27313


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,799822,58906,327549,317957,16437,199005,131700,330705,43577
2,814326,18156,396613,323497,29579,143146,146930,290076,16022
3,793143,42464,441217,223514,12891,209117,169748,378865,37974
4,796191,30753,372894,322945,16148,207887,136482,344369,20827
5,800333,6327,501593,113381,135452,53537,43719,97256,9176
6,791615,50894,381792,277058,17203,184923,138996,323919,29544
7,786489,55355,319818,339491,14978,151456,150659,302115,22162
8,773498,26044,517696,141903,15044,148392,230730,379122,35464
9,796085,28262,557375,132507,6198,173355,253619,426974,34091


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop
district,,,,,,,,,
1,779557,30120,523975,157590,7180,146977,225751,372728,24339
2,802396,25120,528671,176960,10816,154612,223716,378328,22989
3,783238,42533,238672,442427,17591,124668,82234,206902,16319
4,777750,6201,469542,114116,146633,55642,48627,104269,8725
5,801061,25337,403757,309073,12211,192005,164766,356771,17662
6,806583,43631,436698,236714,10588,173044,181755,354799,41686
7,779802,68244,310605,330293,16390,214583,104258,318841,22862
8,811285,36911,427928,259925,25209,195243,186765,382008,24252
9,809830,39064,476699,165155,17312,214044,184711,398755,70003


In [270]:
def is_opportunity(total_pop ,african_american_pop, latino_pop, asain_pop, native_american_pop):
    if(african_american_pop / total_pop >= .5):
        return True
    if(latino_pop / total_pop >= .5):
        return True
    if(asain_pop / total_pop >= .5):
        return True
    if(native_american_pop / total_pop >= .5):
        return True
    return False

In [271]:
def election(dem, rep):
    if(dem > rep):
        return 'democrat'
    return 'republican'

In [272]:
def majority_race(total_pop ,african_american_pop, latino_pop, asian_pop, native_american_pop, ):
    if(african_american_pop / total_pop >= .5):
        return "africanAmerican"
    if(latino_pop / total_pop >= .5):
        return True
    if(asian_pop / total_pop >= .5):
        return True
    if(native_american_pop / total_pop >= .5):
        return True
    return False

In [273]:
def is_safe(winner, dem, rep, total):
    if(winner == 'democrat'):
        if(dem/total > .6):
            return True
        return False
    else:
        if(rep/total > .6):
            return True
        return False

In [274]:
for plan in ensamble:
    plan = ensamble[plan]
    plan['opportunityDistrict'] = plan.apply(lambda x: is_opportunity(x.totalPop, x.africanAmericanPop, x.latinoPop, x.asianPop, x.nativeAmericanPop), axis=1)
    plan['winningParty'] = plan.apply(lambda x: election(x.demVotes, x.repVotes), axis=1)
    plan['safeDistrict'] = plan.apply(lambda x: is_safe(x.winningParty, x.demVotes, x.repVotes, x.totalVotes), axis=1)

In [275]:
display(ensamble[5000])

,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,opportunityDistrict,winningParty,safeDistrict
district,,,,,,,,,,,,
1,799822,58906,327549,317957,16437,199005,131700,330705,43577,False,democrat,True
2,814326,18156,396613,323497,29579,143146,146930,290076,16022,False,republican,False
3,793143,42464,441217,223514,12891,209117,169748,378865,37974,False,democrat,False
4,796191,30753,372894,322945,16148,207887,136482,344369,20827,False,democrat,True
5,800333,6327,501593,113381,135452,53537,43719,97256,9176,False,democrat,False
6,791615,50894,381792,277058,17203,184923,138996,323919,29544,False,democrat,False
7,786489,55355,319818,339491,14978,151456,150659,302115,22162,False,democrat,False
8,773498,26044,517696,141903,15044,148392,230730,379122,35464,False,republican,True
9,796085,28262,557375,132507,6198,173355,253619,426974,34091,False,republican,False


SMD Ensamble Analysis

In [276]:
#seat share
total_dem_seats = 0
min_dem_seats = float('inf')
max_dem_seats = float('-inf')
max_dem_index = -1

total_rep_seats = 0
min_rep_seats = float('inf')
max_rep_seats = float('-inf')
max_rep_index = -1
total_seats = 0

for plan_index in ensamble:
    plan = ensamble[plan_index]
    counts = plan['winningParty'].value_counts()
    counts = counts.to_dict()
    if("republican" not in counts):
        counts["republican"] = 0
    if("democrat" not in counts):
        counts["democrat"] = 0
    total_dem_seats = total_dem_seats + counts["democrat"]
    total_rep_seats = total_rep_seats + counts["republican"]
    total_seats = total_seats + counts["republican"] + counts["democrat"]
    min_dem_seats = min(min_dem_seats, counts["democrat"])
    if(max_dem_seats < counts["democrat"]):
        max_dem_seats = counts["democrat"]
        max_dem_index = plan_index
    min_rep_seats = min(min_rep_seats, counts["republican"])
    if(max_rep_seats < counts["republican"]):
        max_rep_seats = counts["republican"]
        max_rep_index = plan_index

avg_dem_seats = float(total_dem_seats)/len(ensamble)
display(total_dem_seats)
display(len(ensamble))
display(avg_dem_seats)
avg_rep_seats = float(total_rep_seats)/len(ensamble)

53358

10000

5.3358

In [277]:
oppertunity_districts = 0
total_districts = 0
min_opportunity_districts = float('inf')
max_opportunity_districts = float('-inf')
min_opportunity_index = -1
max_opportunity_index = -1

for plan_index in ensamble:
    plan = ensamble[plan_index]
    counts = plan['opportunityDistrict'].value_counts()
    counts = counts.to_dict()
    if(True not in counts):
        counts[True] = 0
    if(False not in counts):
        counts[False] = 0
    oppertunity_districts = oppertunity_districts + counts[True]
    total_districts = total_districts + counts[True] + counts[False]
    if(min_opportunity_districts > counts[True]):
        min_opportunity_districts = counts[True]
        min_opportunity_index = plan_index
    if(max_opportunity_districts < counts[True]):
        max_opportunity_districts = counts[True]
        max_opportunity_index = plan_index

display(oppertunity_districts)
display(total_districts)
avg_opportunity_districts = float(oppertunity_districts) / len(ensamble)

7414

90000

In [278]:
safe_districts = 0
total_districts = 0
min_safe_districts = float('inf')
max_safe_districts = float('-inf')
min_safe_index = -1
max_safe_index = -1

for plan_index in ensamble:
    plan = ensamble[plan_index]
    counts = plan['safeDistrict'].value_counts()
    counts = counts.to_dict()
    if(True not in counts):
        counts[True] = 0
    if(False not in counts):
        counts[False] = 0
    safe_districts = safe_districts + counts[True]
    total_districts = total_districts + counts[True] + counts[False]
    if(min_safe_districts > counts[True]):
        min_safe_districts = counts[True]
        min_safe_index = plan_index
    if(max_safe_districts < counts[True]):
        max_safe_districts = counts[True]
        max_safe_index = plan_index

avg_safe_districts = float(safe_districts) / len(ensamble)
display(avg_safe_districts)

1.9585

SMD Sample Plan Compilation

In [279]:
def generate_representative(party, vote_recieved, total_votes):
    return {
        "party": party,
        "votesReceived": int(vote_recieved),
        "wonBy": int(vote_recieved - (total_votes - vote_recieved))
    }

In [280]:
def generate_election(district):
    if(district.winningParty == "democrat"):
        winning_party = "democrat"
        winning_votes = int(district.demVotes)
        losing_party = "republican"
        losing_votes = int(district.repVotes)
    else:
        winning_party = "republican"
        winning_votes = int(district.repVotes)
        losing_party = "democrat"
        losing_votes = int(district.demVotes)
    winner = [generate_representative(winning_party, winning_votes, district.totalVotes)]
    losers = [
        generate_representative(losing_party, losing_votes, district.totalVotes)
    ]
    return {
        "winners": winner,
        "losers": losers
    }


In [281]:
def generate_district_elections(plan):
    election = []
    for district_id in plan.index:
        election.append(generate_election(plan.loc[district_id]))
    return election

In [282]:
current_district_plan = precincts.copy(deep = True)
current_district_plan.drop(["index", "precinct"], axis = 1, inplace=True)
current_district_plan = current_district_plan.dissolve(by = 'district', aggfunc = 'sum')
current_district_plan['opportunityDistrict'] = current_district_plan.apply(lambda x: is_opportunity(x.totalPop, x.africanAmericanPop, x.latinoPop, x.asianPop, x.nativeAmericanPop), axis=1)
current_district_plan['winningParty'] = current_district_plan.apply(lambda x: election(x.demVotes, x.repVotes), axis=1)
current_district_plan['safeDistrict'] = current_district_plan.apply(lambda x: is_safe(x.winningParty, x.demVotes, x.repVotes, x.totalVotes), axis=1)

In [283]:
def generate_sample_plan_json(district_plan, is_smd, tag): 
    file_name = STATE_NAME+"_"+("smd" if is_smd else "mmd")+"_"+tag+".json"
    district_plan.to_file('server/sample_plans/'+file_name, driver='GeoJSON', index = True)
    return {
        "geojson": "sample_plans/"+file_name,
        "smd": is_smd,
        "representatives": generate_district_elections(district_plan),
        "tag": tag
            }

In [284]:
def generate_sample_plan(plan_index, is_smd = True):
    district_plan = precincts.copy(deep = True)
    district_plan['district'] = district_plan['index'].apply(lambda id: map_to_district(id, str(plan_index)))
    district_plan.drop(["index", "precinct"], axis = 1, inplace=True)
    district_plan = district_plan.dissolve(by = 'district', aggfunc = 'sum')
    district_plan['opportunityDistrict'] = district_plan.apply(lambda x: is_opportunity(x.totalPop, x.africanAmericanPop, x.latinoPop, x.asianPop, x.nativeAmericanPop), axis=1)
    district_plan['winningParty'] = district_plan.apply(lambda x: election(x.demVotes, x.repVotes), axis=1)
    district_plan['safeDistrict'] = district_plan.apply(lambda x: is_safe(x.winningParty, x.demVotes, x.repVotes, x.totalVotes), axis=1)
    return district_plan
    

In [285]:
sample_smd_plans = [
    generate_sample_plan_json(current_district_plan, True, "current"),
    generate_sample_plan_json(generate_sample_plan(max_dem_index), True, "extremeDemocrat"),
    generate_sample_plan_json(generate_sample_plan(max_rep_index), True, "extremeRepublican"),
    generate_sample_plan_json(generate_sample_plan(max_opportunity_index), True, "maximumOpportunityDistricts"),
    generate_sample_plan_json(generate_sample_plan(min_opportunity_index), True, "minimumOpportunityDistricts"),
    generate_sample_plan_json(generate_sample_plan(max_safe_index), True, "maximumSafeDistricts"),
    generate_sample_plan_json(generate_sample_plan(min_safe_index), True, "minimumSafeDistricts")
]

SMD Box and Whisker Generation

In [286]:
def box_and_whiskers_json(min, first_quartile, median, third_quartile, max, current_district_value):
    return {
        "firstQuartile": first_quartile,
        "max": max,
        "median": median,
        "min": min,
        "thirdQuartile": third_quartile,
        "currentValue": current_district_value
      }


In [287]:
def generate_box_and_whisker(target_demographic, current_district_plan, ensamble, total_demographic, district_count = SEATS):
    ensamble_population = []
    for district_id in range(0, district_count):
        ensamble_population.append([])
    for plan_id in ensamble:
        populations = ensamble[plan_id][target_demographic].tolist()
        total_populations = ensamble[plan_id][total_demographic].tolist()
        for index in range(0, len(populations)):
            populations[index] = float(populations[index])/total_populations[index]
        populations.sort()
        for district_id in range(0, len(ensamble_population)):
            ensamble_population[district_id].append(populations[district_id])

    current_district_populations = current_district_plan[target_demographic].tolist()
    current_district_total_populations = current_district_plan[total_demographic].tolist()
    for index in range(0, len(current_district_populations)):
        current_district_populations[index] = float(current_district_populations[index])/current_district_total_populations[index]
    current_district_populations.sort()
    
    box_and_whisker = []
    for district_id in range(0, district_count):
        box_and_whisker.append({})
    for district_id in range(0, len(ensamble_population)):
        minimum = min(ensamble_population[district_id])
        maxumum = max(ensamble_population[district_id])
        quartiles = statistics.quantiles(ensamble_population[district_id])
        median = quartiles[1]
        first_quartile = quartiles[0]
        third_quartile = quartiles[2]
        box_and_whisker[district_id] = box_and_whiskers_json(minimum, first_quartile, median, third_quartile, maxumum, current_district_populations[district_id])
    return box_and_whisker

In [288]:
smd_box_and_whiskers = {
    "black": generate_box_and_whisker("africanAmericanPop", current_district_plan, ensamble, "totalPop"),
    "white": generate_box_and_whisker("caucasianPop", current_district_plan, ensamble, "totalPop"),
    "latino": generate_box_and_whisker("latinoPop", current_district_plan, ensamble, "totalPop"),
    "nativeAmerican": generate_box_and_whisker("nativeAmericanPop", current_district_plan, ensamble, "totalPop"),
    "asian": generate_box_and_whisker("asianPop", current_district_plan, ensamble, "totalPop"),
    "democrat": generate_box_and_whisker("demVotes", current_district_plan, ensamble, "totalVotes"),
    "republican": generate_box_and_whisker("repVotes", current_district_plan, ensamble, "totalVotes"),
}

In [289]:
smd_ensamble_data = {
    "minOpportunityReps": int(min_opportunity_districts),
    'maxOpportunityReps': int(max_opportunity_districts),
    'avgOpportunityReps': float(avg_opportunity_districts),
    'minSafeDistricts': int(min_safe_districts),
    'maxSafeDistricts': int(max_safe_districts),
    'avgSafeDistrits': float(avg_safe_districts),
    'maxDemocratReps': int(max_dem_seats),
    'minDemocratReps': int(min_dem_seats),
    'avgDemocratReps': float(avg_dem_seats),
    'maxRepublicanReps': int(max_rep_seats),
    'minRepublicanReps': int(min_rep_seats),
    'avgRepublicanReps': float(avg_rep_seats)
}

MMD Ensamble Dataframe Generation

In [290]:
def mmd_map_to_district(precinct_id, plan_id):
    plan = mmd_ensamble_ids[plan_id]
    for district_id in plan:
        if(precinct_id in plan[district_id]['nodes']):
            return int(district_id)
    return -1

In [291]:
def map_to_seats(district_id, plan_id):
    plan = mmd_ensamble_ids[plan_id]
    return plan[str(district_id)]['size']

In [292]:
for plan_id in mmd_ensamble_ids:
    district_plan = precincts.copy(deep = True)
    district_plan['district'] = district_plan['index'].apply(lambda id: mmd_map_to_district(id, plan_id))
    district_plan.drop(["index", "precinct", "geometry"], axis = 1, inplace=True)
    district_plan = district_plan.groupby('district').aggregate(sum)
    district_plan['seats'] = district_plan.apply(lambda x: map_to_seats(x.name, plan_id), axis = 1)
    district_plan['seats'] = district_plan['seats'].astype(str)
    mmd_ensamble[int(plan_id)] = district_plan
display(mmd_ensamble[0])

,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats
district,,,,,,,,,,
1,3191931,169194,1696937,1017667,44871,655371,635943,1291314,122830,4
2,3959571,147967,2119610,1174586,219059,815447,766640,1582087,126007,5


MMD Ensamble Analysis

In [293]:
def get_opportunity_reps(plan_id, district_id):
    return mmd_election[str(plan_id)][str(district_id)]['numMinorityReps']

In [294]:
def get_winners(plan_id, district_id, party):
    winners = mmd_election[str(plan_id)][str(district_id)]['winners']
    winner_count = 0
    for rep in winners:
        if(rep[0].split('_')[0] == party):
            winner_count = winner_count + 1
    return winner_count

In [295]:
def get_safe_dist(plan_id, district_id):
    return str(district_id) in mmd_safe_dists[str(plan_id)]

In [296]:
for plan_id in mmd_ensamble_ids:
    plan = mmd_ensamble[int(plan_id)]
    plan['numOpportunity'] = plan.apply(lambda x: get_opportunity_reps(plan_id, x.name), axis = 1)
    plan['democratReps'] = plan.apply(lambda x: get_winners(plan_id, x.name, 'democrat'), axis = 1)
    plan['republicanReps'] = plan.apply(lambda x: get_winners(plan_id, x.name, 'republican'), axis = 1)
    plan['safeDistrict'] = plan.apply(lambda x: get_safe_dist(plan_id, x.name), axis = 1)
display(mmd_ensamble[0])
display(mmd_ensamble[1000])
display(mmd_ensamble[7000])
    


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats,numOpportunity,democratReps,republicanReps,safeDistrict
district,,,,,,,,,,,,,,
1,3191931,169194,1696937,1017667,44871,655371,635943,1291314,122830,4,2,2,2,False
2,3959571,147967,2119610,1174586,219059,815447,766640,1582087,126007,5,2,3,2,False


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats,numOpportunity,democratReps,republicanReps,safeDistrict
district,,,,,,,,,,,,,,
1,3185499,88558,1755542,919951,202054,557574,546360,1103934,84264,4,2,2,2,False
2,3966003,228603,2061005,1272302,61876,913244,856223,1769467,164573,5,2,3,2,False


,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats,numOpportunity,democratReps,republicanReps,safeDistrict
district,,,,,,,,,,,,,,
1,3116443,115826,1679821,990451,83516,672075,692760,1364835,110973,4,2,2,2,False
2,4035059,201335,2136726,1201802,180414,798743,709823,1508566,137864,5,2,3,2,False


In [297]:
#seat share
total_dem_seats = 0
min_dem_seats = float('inf')
max_dem_seats = float('-inf')
max_dem_index = -1

total_rep_seats = 0
min_rep_seats = float('inf')
max_rep_seats = float('-inf')
max_rep_index = -1
total_seats = 0

for plan_index in mmd_ensamble:
    plan = mmd_ensamble[plan_index]
    dem_seats = plan['democratReps'].sum()
    rep_seats = plan['republicanReps'].sum()

    total_dem_seats = total_dem_seats + dem_seats
    total_rep_seats = total_rep_seats + rep_seats
    total_seats = total_seats + rep_seats + dem_seats
    min_dem_seats = min(min_dem_seats, dem_seats)
    if(max_dem_seats < dem_seats):
        max_dem_seats = dem_seats
        max_dem_index = plan_index
    min_rep_seats = min(min_rep_seats, rep_seats)
    if(max_rep_seats < rep_seats):
        max_rep_seats = rep_seats
        max_rep_index = plan_index

avg_dem_seats = float(total_dem_seats)/len(ensamble)
avg_rep_seats = float(total_rep_seats)/len(ensamble)
display(mmd_ensamble[0])

,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats,numOpportunity,democratReps,republicanReps,safeDistrict
district,,,,,,,,,,,,,,
1,3191931,169194,1696937,1017667,44871,655371,635943,1291314,122830,4,2,2,2,False
2,3959571,147967,2119610,1174586,219059,815447,766640,1582087,126007,5,2,3,2,False


In [298]:
oppertunity_districts = 0
total_districts = 0
min_opportunity_districts = float('inf')
max_opportunity_districts = float('-inf')
min_opportunity_index = -1
max_opportunity_index = -1

for plan_index in mmd_ensamble:
    plan = mmd_ensamble[plan_index]
    opportunity_reps = plan['numOpportunity'].sum()
    oppertunity_districts = oppertunity_districts + opportunity_reps
    total_districts = total_districts + SEATS
    if(min_opportunity_districts > opportunity_reps):
        min_opportunity_districts = opportunity_reps
        min_opportunity_index = plan_index
    if(max_opportunity_districts < opportunity_reps):
        max_opportunity_districts = opportunity_reps
        max_opportunity_index = plan_index

display(float(oppertunity_districts))
display(total_districts)
avg_opportunity_districts = float(oppertunity_districts) / len(mmd_ensamble)
display(avg_opportunity_districts)

41118.0

90000

4.1118

In [299]:
safe_districts = 0
total_districts = 0
min_safe_districts = float('inf')
max_safe_districts = float('-inf')
min_safe_index = -1
max_safe_index = -1

for plan_index in mmd_ensamble:
    plan = mmd_ensamble[plan_index]
    counts = plan['safeDistrict'].value_counts()
    counts = counts.to_dict()
    if(True not in counts):
        counts[True] = 0
    if(False not in counts):
        counts[False] = 0
    safe_districts = safe_districts + counts[True]
    total_districts = total_districts + counts[True] + counts[False]
    if(min_safe_districts > counts[True]):
        min_safe_districts = counts[True]
        min_safe_index = plan_index
    if(max_safe_districts < counts[True]):
        max_safe_districts = counts[True]
        max_safe_index = plan_index

avg_safe_districts = float(safe_districts) / len(mmd_ensamble)
display(avg_safe_districts)

0.2068

In [300]:
display(avg_safe_districts)
display(avg_opportunity_districts)
display(avg_dem_seats)
display(avg_rep_seats)
display(mmd_ensamble[4895])

0.2068

4.1118

4.7056

4.2944

,totalPop,africanAmericanPop,caucasianPop,latinoPop,nativeAmericanPop,demVotes,repVotes,totalVotes,asianPop,seats,numOpportunity,democratReps,republicanReps,safeDistrict
district,,,,,,,,,,,,,,
1,3166521,99918,1807217,864651,188649,590042,612523,1202565,67747,4,2,2,2,False
2,3984981,217243,2009330,1327602,75281,880776,790060,1670836,181090,5,2,3,2,False


MMD Sample Plan Compilation

In [301]:
average_mmd_id = 4895

average_plan = precincts.copy(deep = True)
average_plan['district'] = average_plan['index'].apply(lambda id: mmd_map_to_district(id, str(average_mmd_id)))
average_plan.drop(["index", "precinct"], axis = 1, inplace=True)
average_plan = average_plan.dissolve(by = 'district', aggfunc = 'sum')
average_plan['numOpportunity'] = average_plan.apply(lambda x: get_opportunity_reps(average_mmd_id, x.name), axis = 1)
average_plan['democratReps'] = average_plan.apply(lambda x: get_winners(average_mmd_id, x.name, 'democrat'), axis = 1)
average_plan['republicanReps'] = average_plan.apply(lambda x: get_winners(average_mmd_id, x.name, 'republican'), axis = 1)
average_plan['safeDistrict'] = average_plan.apply(lambda x: get_safe_dist(average_mmd_id, x.name), axis = 1)

In [302]:
def generate_mmd_representative(party, vote_recieved):
    return {
        "party": party,
        "votesReceived": int(vote_recieved),
    }

In [303]:
def generate_mmd_election(district_id, plan_id):
    winners = []
    losers = []
    winners_list = mmd_election[str(plan_id)][str(district_id)]['winners']
    losers_list = mmd_election[str(plan_id)][str(district_id)]['losers']
    for results in winners_list:
        winners.append(generate_mmd_representative(results[0].split('_')[0], results[1]))
    for results in losers_list:
        losers.append(generate_mmd_representative(results[0].split('_')[0], results[1]))
    

    return {
        "winners": winners,
        "losers": losers
    }


In [304]:
def generate_district_mmd_elections(plan, plan_id):
    election = []
    for district_id in plan.index:
        election.append(generate_mmd_election(district_id, plan_id))
    return election

In [305]:
def generate_sample_mmd_plan_json(district_plan, plan_id, is_smd, tag): 
    file_name = STATE_NAME+"_"+("smd" if is_smd else "mmd")+"_"+tag+".json"
    district_plan.to_file('server/sample_plans/'+file_name, driver='GeoJSON', index = True)
    return {
        "geojson": "sample_plans/"+file_name,
        "smd": is_smd,
        "representatives": generate_district_mmd_elections(district_plan, plan_id),
        "tag": tag
            }

In [306]:
def generate_sample_mmd_plan(plan_index, is_smd = False):
    district_plan = precincts.copy(deep = True)
    district_plan['district'] = district_plan['index'].apply(lambda id: mmd_map_to_district(id, str(plan_index)))
    district_plan.drop(["index", "precinct"], axis = 1, inplace=True)
    district_plan = district_plan.dissolve(by = 'district', aggfunc = 'sum')
    district_plan['numOpportunity'] = district_plan.apply(lambda x: get_opportunity_reps(plan_index, x.name), axis = 1)
    district_plan['democratReps'] = district_plan.apply(lambda x: get_winners(plan_index, x.name, 'democrat'), axis = 1)
    district_plan['republicanReps'] = district_plan.apply(lambda x: get_winners(plan_index, x.name, 'republican'), axis = 1)
    district_plan['safeDistrict'] = district_plan.apply(lambda x: get_safe_dist(plan_index, x.name), axis = 1)
    return district_plan

In [307]:
sample_mmd_plans = [
    generate_sample_mmd_plan_json(average_plan, str(average_mmd_id), False, "average"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(max_dem_index), max_dem_index, False, "extremeDemocrat"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(max_rep_index), max_rep_index, False, "extremeRepublican"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(max_opportunity_index), max_opportunity_index, False, "maximumOpportunityDistricts"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(min_opportunity_index), min_opportunity_index, False, "minimumOpportunityDistricts"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(max_safe_index), max_safe_index, False, "maximumSafeDistricts"),
    generate_sample_mmd_plan_json(generate_sample_mmd_plan(min_safe_index), min_safe_index, False, "minimumSafeDistricts")
]

In [308]:
def is_config(plan, config_list):
    config = {
        3: 0,
        4: 0,
        5: 0
    }
    for seat_num in config_list:
        config[seat_num] = config[seat_num] + 1

    #display(plan)
    counts = plan['seats'].value_counts()
    counts = counts.to_dict()
    if('3' not in counts):
        counts['3'] = 0
    if('4' not in counts):
        counts['4'] = 0
    if('5' not in counts):
        counts['5'] = 0

    for seat_num in config:
        if(config[seat_num] != counts[str(seat_num)]):
            return False
    return True

In [309]:
mmd_ensamble_config = {}
for config_id in range(0, len(MMD_PATTERN)):
    mmd_ensamble_config[config_id] = []
for config_id in range(0, len(MMD_PATTERN)):
    for plan_id in mmd_ensamble:
        if(is_config(mmd_ensamble[plan_id], MMD_PATTERN[config_id])):
            mmd_ensamble_config[config_id].append(mmd_ensamble[plan_id])

In [310]:
display(len(mmd_ensamble_config))
display(len(mmd_ensamble_config[0]))
#display(len(mmd_ensamble_config[1]))
display(len(mmd_ensamble))

2

1989

10000

In [311]:
def generate_dummy_mmd_box():
    return {
        "firstQuartile": 1,
        "max": 1,
        "median": 1,
        "min": 1,
        "thirdQuartile": 1
    }

In [312]:
def box_and_whiskers_mmd_json(min, first_quartile, median, third_quartile, max):
    return {
        "firstQuartile": first_quartile,
        "max": max,
        "median": median,
        "min": min,
        "thirdQuartile": third_quartile,
      }

In [313]:
def generate_mmd_box_and_whisker(target_demographic, ensamble, district_count, total_demograhpic):
    ensamble_population = []
    for district_id in range(0, district_count):
        ensamble_population.append([])
    for plan in ensamble:
        populations_by_dist = plan[target_demographic].tolist()
        total_populations_by_dist = plan[total_demograhpic].tolist()
        seats_by_dist = plan["seats"].tolist()
        populations = []
        for district_id in range(0, len(seats_by_dist)):
            percent = float(populations_by_dist[district_id]) / total_populations_by_dist[district_id]
            for i in range(0, int(seats_by_dist[district_id])):
                populations.append(percent)

        populations.sort()
        for district_id in range(0, len(ensamble_population)):
            ensamble_population[district_id].append(populations[district_id])

    
    box_and_whisker = []
    for district_id in range(0, district_count):
        box_and_whisker.append({})
    for district_id in range(0, len(ensamble_population)):
        if(len(ensamble_population[district_id]) == 1):
            minimum = maxumum = median = first_quartile = third_quartile = ensamble_population[district_id][0]
        else:
            minimum = min(ensamble_population[district_id])
            maxumum = max(ensamble_population[district_id])
            quartiles = statistics.quantiles(ensamble_population[district_id])
            median = quartiles[1]
            first_quartile = quartiles[0]
            third_quartile = quartiles[2]
        box_and_whisker[district_id] = box_and_whiskers_mmd_json(minimum, first_quartile, median, third_quartile, maxumum)
    return box_and_whisker

In [314]:
def generate_dummy_mmd_box_tag(tag, num_boxes):
    return {
        "tag": str(tag),
        "numberDistricts": num_boxes,
        "black": generate_dummy_mmd_box(),
        "white": generate_dummy_mmd_box(),
        "latino": generate_dummy_mmd_box(),
        "nativeAmerican": generate_dummy_mmd_box(),
        "asian": generate_dummy_mmd_box(),
        "democrat": generate_dummy_mmd_box(),
        "republican": generate_dummy_mmd_box()
    }

In [315]:
def generate_mmd_box_tag(tag, num_boxes, sub_ensamble):
    return {
        "tag": str(tag),
        "numberDistricts": num_boxes,
        "black": generate_mmd_box_and_whisker("africanAmericanPop", sub_ensamble, SEATS, "totalPop"),
        "white": generate_mmd_box_and_whisker("caucasianPop", sub_ensamble, SEATS, "totalPop"),
        "latino": generate_mmd_box_and_whisker("latinoPop", sub_ensamble, SEATS, "totalPop"),
        "nativeAmerican": generate_mmd_box_and_whisker("nativeAmericanPop", sub_ensamble, SEATS, "totalPop"),
        "asian": generate_mmd_box_and_whisker("asianPop", sub_ensamble, SEATS, "totalPop"),
        "democrat": generate_mmd_box_and_whisker("demVotes", sub_ensamble, SEATS, "totalVotes"),
        "republican": generate_mmd_box_and_whisker("repVotes", sub_ensamble, SEATS, "totalVotes")
    }

In [316]:
mmd_box_and_whiskers = []
for config_id in range(0, len(MMD_PATTERN)):
    if(len(mmd_ensamble_config[config_id]) <= 0):
        continue
    mmd_box_and_whiskers.append(generate_mmd_box_tag(MMD_PATTERN[config_id], len(MMD_PATTERN[config_id]), mmd_ensamble_config[config_id]))
display(mmd_box_and_whiskers)
display(len(mmd_box_and_whiskers))

[{'tag': '[3, 3, 3]',
  'numberDistricts': 3,
  'black': [{'firstQuartile': 0.02506750450476831,
    'max': 0.0435466185691487,
    'median': 0.029802959204634504,
    'min': 0.01792987987894423,
    'thirdQuartile': 0.0335714706172192},
   {'firstQuartile': 0.02506750450476831,
    'max': 0.0435466185691487,
    'median': 0.029802959204634504,
    'min': 0.01792987987894423,
    'thirdQuartile': 0.0335714706172192},
   {'firstQuartile': 0.02506750450476831,
    'max': 0.0435466185691487,
    'median': 0.029802959204634504,
    'min': 0.01792987987894423,
    'thirdQuartile': 0.0335714706172192},
   {'firstQuartile': 0.039154540056814444,
    'max': 0.05586494624269765,
    'median': 0.0435574973856364,
    'min': 0.029876001795002514,
    'thirdQuartile': 0.047681854129886465},
   {'firstQuartile': 0.039154540056814444,
    'max': 0.05586494624269765,
    'median': 0.0435574973856364,
    'min': 0.029876001795002514,
    'thirdQuartile': 0.047681854129886465},
   {'firstQuartile': 0.0

2

In [317]:
mmd_ensamble_data = {
    "minOpportunityReps": int(min_opportunity_districts),
    'maxOpportunityReps': int(max_opportunity_districts),
    'avgOpportunityReps': float(avg_opportunity_districts),
    'minSafeDistricts': int(min_safe_districts),
    'maxSafeDistricts': int(max_safe_districts),
    'avgSafeDistrits': float(avg_safe_districts),
    'maxDemocratReps': int(max_dem_seats),
    'minDemocratReps': int(min_dem_seats),
    'avgDemocratReps': float(avg_dem_seats),
    'maxRepublicanReps': int(max_rep_seats),
    'minRepublicanReps': int(min_rep_seats),
    'avgRepublicanReps': float(avg_rep_seats)

}

Data Export

In [318]:
state_demographics = {
    "blackPopulation": int(BLACK_POPULATION),
    "democratPopulation": int(DEMOCRAT_VOTES),
    "hispanicPopulation": int(LATINO_POPULATION),
    "asianPopulation": int(ASIAN_POPULATION),
    "nativeAmericanPopulation": int(NATIVE_AMERICAN_POPULATION),
    "republicanPopulation": int(REPUBLICAN_VOTES),
    "whitePopulation": int(WHITE_POPULATION),
    "totalPopulation": int(TOTAL_POPULATION)
}

In [319]:
state_vote_share = {
    'democrateVotes' : int(DEMOCRAT_VOTES),
    'republicanVotes' : int(REPUBLICAN_VOTES)
}

In [320]:
state = [{
  "demographics": state_demographics,
  "voteShare": state_vote_share,
  "smdEnsembleData": smd_ensamble_data,
  "mmdEnsembleData": mmd_ensamble_data,
  "mmdPlans": sample_mmd_plans,
  "name": STATE_NAME,
  "numSeats": SEATS,
  "smdPlans": sample_smd_plans,
  'mmdBoxAndWhiskers': mmd_box_and_whiskers,
  'smdBoxAndWhiskers': smd_box_and_whiskers
}]

In [321]:
with open('server/server'+STATE_NAME_UPPER+'.json', 'w', encoding='utf-8') as file:
    json.dump(state, file, ensure_ascii=False, indent=4)